In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db = sqlite3.connect("Monarchs1995.db")

In [3]:
df = pd.read_csv("1995 Season Summary.csv")

        #convert pandas to SQL table
df.to_sql("monarchs1995", db, if_exists="replace")

os.getcwd()

C:\Users\Arink\Anaconda3\lib\site-packages\pandas\core\generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


'C:\\Users\\Arink\\Desktop\\github\\CodeLou\\Monarchs\\Data'

In [4]:
schema = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
schema

,type,name,tbl_name,rootpage,sql
0,index,ix_monarchs1995_index,monarchs1995,3,"CREATE INDEX ""ix_monarchs1995_index""ON ""monarc..."
1,table,monarchs1995,monarchs1995,2,"CREATE TABLE ""monarchs1995"" (\n""index"" INTEGER..."


## Checking DataType

In [5]:
data_type = pd.read_sql('SELECT * FROM PRAGMA_TABLE_INFO("monarchs1995")', db)
data_type

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Tag No.,TEXT,0,None,0
2,2,Tagger,TEXT,0,None,0
3,3,Mon. Sex,TEXT,0,None,0
4,4,Where Tagged,TEXT,0,None,0
5,5,Date Tagged,TEXT,0,None,0
6,6,Date Recovered,TEXT,0,None,0
7,7,Where Recovered,TEXT,0,None,0
8,8,Observer,TEXT,0,None,0
9,9,Interval,TEXT,0,None,0


### Drop tagger and reporter identification

In [6]:
df.drop(["Tagger", "Mon. Sex", "Observer", "Interval"], axis = 1, inplace = True)
df.head(10)

,Tag No.,Where Tagged,Date Tagged,Date Recovered,Where Recovered,Miles
0,AU 646,"Keene, NY",19 Sep,Feb 96,"El Rosario colony, \nMexico",2580.0
1,BW 003,"Huntingtown, \nMD",14 Sep,Dec\n1995,"El Rosario\ncolony, Mexico",2285.0
2,AN 008,"West St Paul, MN",4 Sep,1 Mar,"El Rosario\ncolony, Mexico",1990.0
3,AG 802,"Sheldon, IA",NaN,Mar\n1996,"Chinqua, Mexico",1820.0
4,CU 878,"Wamego, KS",23 Sep,Mar 96,"Chinqua, Mexico",1555.0
5,BP 359,"Hays, KS",18 Sep,24 Feb,"Sierra Chinqua, Mexico",1510.0
6,BP 652,"Hays, KS",18 Sep,Mar\n1996,"Chinqua, \nMexico",1510.0
7,BO 299,"Hutchinson, KS",1 Oct,15 Jan,"Herrada or Las Palomas, \nMexico",1500.0
8,CM 355,"Berryville, AR",25 Sep,30 Dec,"El Rosario colony, Mexico",1445.0
9,CJ 954,"Andover, KS",24 Sep,Mar\n1996,"Chinqua, Mexico",1445.0


### Split & rename combined columns in Tagged category

In [7]:
df.loc[df['Where Tagged'].str.contains("\n", na=False), "Where Tagged"] = df.loc[df['Where Tagged'].str.contains("\n", na=False), "Where Tagged"].str.replace("\n", " ")
df2 = df['Where Tagged'].str.split(', ', expand=True, n=1)
df2.columns = ['tag_city', 'Tag State']
df2['Tag State'] = df2['Tag State'].str.strip()
df2.head(10)

,tag_city,Tag State
0,Keene,NY
1,Huntingtown,MD
2,West St Paul,MN
3,Sheldon,IA
4,Wamego,KS
5,Hays,KS
6,Hays,KS
7,Hutchinson,KS
8,Berryville,AR
9,Andover,KS


### Split & Rename combined columns in Reported category

In [8]:
df.loc[df['Where Recovered'].str.contains("\n", na=False), "Where Recovered"] = df.loc[df['Where Recovered'].str.contains("\n", na=False), "Where Recovered"].str.replace("\n", " ")
df3 = df['Where Recovered'].str.split(', ', expand=True, n=1)
df3.columns = ['report_city', 'Report State']
df3['Report State'] = df3['Report State'].str.strip()
df3.loc[df3['Report State'] == 'Mexico', 'Report State'] = 'Michoacán'
df3.head(10)

,report_city,Report State
0,El Rosario colony,Michoacán
1,El Rosario colony,Michoacán
2,El Rosario colony,Michoacán
3,Chinqua,Michoacán
4,Chinqua,Michoacán
5,Sierra Chinqua,Michoacán
6,Chinqua,Michoacán
7,Herrada or Las Palomas,Michoacán
8,El Rosario colony,Michoacán
9,Chinqua,Michoacán


### Create new column Report Country & clean up

In [9]:
add_country = df['Where Recovered'].str.extract("(Mexico)", expand=True)
add_country.columns = ['Report Country']
add_country.loc[add_country['Report Country'] == 'Mexico', 'Report Country'] = 'MEXICO'
add_country.loc[add_country['Report Country'].isnull(), 'Report Country'] = 'USA'
add_country.head(5)

,Report Country
0,MEXICO
1,MEXICO
2,MEXICO
3,MEXICO
4,MEXICO


### Date columns, are in odd order, need to split then put back together 

In [10]:
fix_tag = df['Date Tagged'].str.extract('([a-zA-z]{3})', expand=True)
fix_tag.columns = ['tag_month']
fix_tag['tag_date'] = df['Date Tagged'].str.extract('(.+)[ ]', expand=True)
fix_tag.loc[fix_tag['tag_date'] == '10 May', 'tag_date'] = '10'
fix_tag.loc[(fix_tag['tag_month'] == 'May'), 'tag_year'] = '1996' 
fix_tag.loc[(fix_tag['tag_month'] != 'May'), 'tag_year'] = '1995'
fix_tag['Tag Date'] = fix_tag["tag_month"] +" "+ fix_tag["tag_date"] +" "+ fix_tag["tag_year"]
# fix_tag.drop(['tag_month', 'tag_date', 'tag_year'], axis=1, inplace=True)
fix_tag.head(10)

,tag_month,tag_date,tag_year,Tag Date
0,Sep,19,1995,Sep 19 1995
1,Sep,14,1995,Sep 14 1995
2,Sep,4,1995,Sep 4 1995
3,NaN,NaN,1995,NaN
4,Sep,23,1995,Sep 23 1995
5,Sep,18,1995,Sep 18 1995
6,Sep,18,1995,Sep 18 1995
7,Oct,1,1995,Oct 1 1995
8,Sep,25,1995,Sep 25 1995
9,Sep,24,1995,Sep 24 1995


### Create new table with unneeded columns dropped

In [11]:
df4 = pd.concat([df, df2, df3, add_country, fix_tag], axis=1)
df4.drop(['Where Tagged', 'Where Recovered', 'report_city', 'tag_city'], axis=1, inplace=True)
df5 = df4[['Tag No.','Tag State', 'Tag Date', 'Report State', 'Report Country', 'Miles']]
new_table = df5.copy()
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Miles
0,AU 646,NY,Sep 19 1995,Michoacán,MEXICO,2580.0
1,BW 003,MD,Sep 14 1995,Michoacán,MEXICO,2285.0
2,AN 008,MN,Sep 4 1995,Michoacán,MEXICO,1990.0
3,AG 802,IA,NaN,Michoacán,MEXICO,1820.0
4,CU 878,KS,Sep 23 1995,Michoacán,MEXICO,1555.0
5,BP 359,KS,Sep 18 1995,Michoacán,MEXICO,1510.0
6,BP 652,KS,Sep 18 1995,Michoacán,MEXICO,1510.0
7,BO 299,KS,Oct 1 1995,Michoacán,MEXICO,1500.0
8,CM 355,AR,Sep 25 1995,Michoacán,MEXICO,1445.0
9,CJ 954,KS,Sep 24 1995,Michoacán,MEXICO,1445.0


### Final check for unwanted spaces

In [12]:
# new_table.loc[new_table['Report State'].str.contains(r'\s', na=False), 'Report State'].value_counts()
# new_table.loc[new_table['Tag State'].str.contains(r'\s', na=False), 'Tag State'].value_counts()
# new_table.loc[new_table['Tag No.'].str.contains(r'\s', na=False), 'Tag No.'].value_counts()
# new_table.loc[new_table['Report Country'].str.contains(r'\s', na=False), 'Report Country'].value_counts()

### Convert date columns to DateTime datatypes

In [13]:
new_table["Tag Date"]= pd.to_datetime(new_table["Tag Date"]) 
#new_table["Report Date"]= pd.to_datetime(new_table["Report Date"]) 
new_table.dtypes

Tag No.                   object
Tag State                 object
Tag Date          datetime64[ns]
Report State              object
Report Country            object
Miles                    float64
dtype: object

In [14]:
del df2, df3, df4, df5, add_country

### Export cleaned data to csv.

In [15]:
#new_table.to_csv("Monarchs1995.csv")